# Data Scraping

In [7]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import nltk
import pymongo

In [11]:
import numpy as np

In [102]:
import pandas as pd

In [47]:
!python -m nltk.downloader all

C:\anaconda\envs\NLP\lib\runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Nishant
[nltk_data]    |     Jain\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\Nishant
[nltk_data]    |     Jain\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\Nishant
[nltk_data]    |     Jain\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to C:\Users\Nishant
[nltk_data]    |     Jain\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.z

In [5]:
url = "https://www.moneycontrol.com/"
req1 = requests.get(url)

mnc = req1.content

soup_mnc = BeautifulSoup(mnc)

mnc_links = soup_mnc.find_all('a')

In [14]:
links = []
valid = []
for i in range(len(mnc_links)):
    try:
        link = mnc_links[i]['href']
    except:
        continue
    if(link.startswith("https://www.moneycontrol.com/news")):
        if(len(link)>100):
            links.append(link)
            valid.append(i)

In [16]:
_ ,unq_index = np.unique(links,return_index=True)

unq_index.sort()

links = np.array(links)[unq_index]
valid = np.array(valid)[unq_index]


### Get Articles Data

In [430]:
def article_to_text(link):
    article = Article(link)
    article.download()
    article.parse()
    return [article.text, article.title]

#### Clean Data via NLP pipeline

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def identity_tokenizer(text):
    return text

In [149]:
def tranform_text(text):
    tokens = nltk.tokenize.word_tokenize(text)

    words = [word for word in tokens if word.isalpha()]

    words = [w for w in words if not w in stop_words]

    stemmed = [porter.stem(word.lower()) for word in words]
    return stemmed

def tfidf(tokenised_text):
    tfdf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)    

    tfdf_vector = tfdf.fit_transform([tokenised_text])

    feature_names = tfdf.get_feature_names()
    dense = tfdf_vector.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    return df

#### Cosine similarity

In [225]:
def cosinesim(link1,link2):  
    doc1_tfidf = tfidf(tranform_text(article_to_text(link1)))

    doc2_tfidf = tfidf(tranform_text(article_to_text(link2)))

    combinde_df = pd.concat([a1,a2], axis=0, ignore_index=True).fillna(0)

    similarity = cosine_similarity([combinde_df.loc[0],combinde_df.loc[1]])
    
    return similarity[0,1]

In [226]:
cosinesim(links[0],links[1])

0.07247518602637545

# Setting up Mongo DB

## MoneyControl

In [1]:
import pymongo

In [275]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["mydatabase"]

In [281]:
mycol = mydb["moneycontrol"]
mycol.drop()

In [277]:
mycol.create_index("link", unique = True)

'link_1'

In [282]:
id_mnc = []
for link in links:
    article_dict = {"link":link,"text": article_to_text(link)[0]}
    x = mycol.insert_one(article_dict)
    id_mnc.append(x.inserted_id)

## Economic Times

In [283]:
url = "https://economictimes.indiatimes.com/"
req2 = requests.get(url)

In [284]:
et = req2.content

soup_et = BeautifulSoup(et)

et_links = soup_et.find_all('a')

In [295]:
et_links[100]

<a data-ga-onclick="23 - href" href="/markets/expert-view/voda-idea-survival-depends-on-whether-promoters-can-invest-more-ss-sirohi/articleshow/77068344.cms" target="_blank">Voda-Idea: Ball's in promoters'court</a>

In [297]:
t = et_links[100]["data-ga-onclick"]

In [368]:
et_links_extracted = []
et_valid = []
for i in range(len(et_links)):
    try:
        link = et_links[i]["data-ga-onclick"]
    except:
        continue
    if(link.endswith("href")):
        if(link[0].isnumeric()):
            if(len(et_links[i]['href'])>75):
                if(et_links[i]['href'].startswith("https")):
                    et_links_extracted.append(et_links[i]['href'])
                else:
                    et_links_extracted.append("https://economictimes.indiatimes.com"+et_links[i]['href'])
                et_valid.append(i)


In [369]:
# pd.DataFrame(zip(et_links_extracted,et_valid))

In [370]:
_ ,unq_index_et = np.unique(et_links_extracted,return_index=True)

unq_index_et.sort()

et_links_extracted = np.array(et_links_extracted)[unq_index_et]
et_valid = np.array(et_valid)[unq_index_et]

#### Store data in db

In [492]:
etcol = mydb["et"]
etcol.drop()

In [493]:
etcol.create_index("link", unique = True)

'link_1'

In [494]:
def article_to_text_bs4(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content)
    try:
        text = soup.find("div", class_="artText").get_text(separator = " ")
    except:
        try:
            text = soup.find("section").get_text(separator = " ")
        except:
            raise ValueError()
    return text

In [495]:
id_et = []
for link in et_links_extracted:
    text = article_to_text(link)[0]
    article_dict = {"link":link,"text": text}
    x = etcol.insert_one(article_dict)
    id_et.append(x.inserted_id)

In [496]:
cursor = etcol.find({})
# for document in cursor: pprint(document)

In [497]:
cursor.count()

<ipython-input-497-168361acaab4>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  cursor.count()


171

In [509]:
cursor = etcol.find({"$where": 'this.text.length < 700'})
id_change = []
for document in cursor: 
    pprint(document)
    id_change.append(document["_id"])

{'_id': ObjectId('5f1704ea828c309483059e06'),
 'link': 'https://economictimes.indiatimes.com/news/economy/policy/electronic-manufacturing-the-problems-with-indias-pli-scheme-and-how-to-fix-it/articleshow/77073565.cms',
 'text': 'By Sanjay KapoorAtmanirbhar Bharat is complementary —not '
         'supplementary — to globalisation. The need to collaborate globally '
         'in search of lasting solutions is probably irreversible for the '
         'overall'}
{'_id': ObjectId('5f1704eb828c309483059e09'),
 'link': 'https://economictimes.indiatimes.com/industry/telecom/telecom-news/huawei-may-lose-airtels-tn-4g-circle-to-ericsson/articleshow/77057167.cms',
 'text': 'New Delhi: Bharti Airtel is likely to replace Huawei with Sweden’s '
         'Ericsson for 4G network expansion and modernisation in the Rest of '
         'Tamil Nadu (RoTN) circle, said people familiar with the matter,'}
{'_id': ObjectId('5f1704f0828c309483059e17'),
 'link': 'https://economictimes.indiatimes.com/news/econom

In [505]:
for ID in id_change:
    etcol.update_one({"_id": pymongo.collection.ObjectId(ID)}, 
                     {"$set":{"text": article_to_text_bs4(etcol.find({"_id": pymongo.collection.ObjectId(ID)})[0]['link'])}})

In [506]:
for ID in id_change:
    pprint(etcol.find({"_id": pymongo.collection.ObjectId(ID)})[0])

{'_id': ObjectId('5f1704ea828c309483059e06'),
 'link': 'https://economictimes.indiatimes.com/news/economy/policy/electronic-manufacturing-the-problems-with-indias-pli-scheme-and-how-to-fix-it/articleshow/77073565.cms',
 'text': 'By Sanjay KapoorAtmanirbhar Bharat is complementary —not '
         'supplementary — to globalisation. The need to collaborate globally '
         'in search of lasting solutions is probably irreversible for the '
         'overall'}
{'_id': ObjectId('5f1704eb828c309483059e09'),
 'link': 'https://economictimes.indiatimes.com/industry/telecom/telecom-news/huawei-may-lose-airtels-tn-4g-circle-to-ericsson/articleshow/77057167.cms',
 'text': 'New Delhi: Bharti Airtel is likely to replace Huawei with Sweden’s '
         'Ericsson for 4G network expansion and modernisation in the Rest of '
         'Tamil Nadu (RoTN) circle, said people familiar with the matter,'}
{'_id': ObjectId('5f1704f0828c309483059e17'),
 'link': 'https://economictimes.indiatimes.com/news/econom